In [4]:
import sys
import pandas as pd
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QVBoxLayout, QPushButton, QLabel, QComboBox,
    QTextEdit, QWidget, QLineEdit, QHBoxLayout, QFileDialog
)
from PyQt5.QtCore import Qt
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
import matplotlib.pyplot as plt

In [ ]:
class MainApp(QMainWindow):
    def __init__(self):
        super().__init__()

        self.setWindowTitle("Анализ данных из CSV")
        self.setGeometry(100, 100, 1200, 600)

        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)

        self.layout = QVBoxLayout(self.central_widget)

        self.load_button = QPushButton("Загрузить CSV")
        self.load_button.clicked.connect(self.load_data)
        self.layout.addWidget(self.load_button)

        self.stats_label = QLabel("Статистика данных:")
        self.layout.addWidget(self.stats_label)
        self.stats_text = QTextEdit()
        self.stats_text.setReadOnly(True)
        self.layout.addWidget(self.stats_text)

        self.graph_type_label = QLabel("Выберите тип графика:")
        self.layout.addWidget(self.graph_type_label)
        self.graph_type_combo = QComboBox()
        self.graph_type_combo.addItems(["Линейный график", "Гистограмма", "Круговая диаграмма"])
        self.graph_type_combo.currentTextChanged.connect(self.update_plot)
        self.layout.addWidget(self.graph_type_combo)

        self.figure = plt.figure()
        self.canvas = FigureCanvas(self.figure)
        self.layout.addWidget(self.canvas)

        # Поля для добавления данных
        self.add_data_layout = QVBoxLayout()
        self.date_field = QLineEdit()
        self.date_field.setPlaceholderText("Дата (YYYY-MM-DD)")
        self.date_field.setText(pd.Timestamp.now().strftime("%Y-%m-%d"))

        self.category_field = QLineEdit()
        self.category_field.setPlaceholderText("Категория (например, A, B, C)")

        self.value1_field = QLineEdit()
        self.value1_field.setPlaceholderText("Value1 (число)")

        self.value2_field = QLineEdit()
        self.value2_field.setPlaceholderText("Value2 (число)")

        self.boolean_combo = QComboBox()
        self.boolean_combo.addItems(["True", "False"])

        self.add_data_button = QPushButton("Добавить данные")
        self.add_data_button.clicked.connect(self.add_data)

        self.add_data_layout.addWidget(QLabel("Добавить данные:"))
        self.add_data_layout.addWidget(self.date_field)
        self.add_data_layout.addWidget(self.category_field)
        self.add_data_layout.addWidget(self.value1_field)
        self.add_data_layout.addWidget(self.value2_field)
        self.add_data_layout.addWidget(self.boolean_combo)
        self.add_data_layout.addWidget(self.add_data_button)
        self.layout.addLayout(self.add_data_layout)

        #Результаты добавления
        self.result_label = QLabel("")
        self.result_label.setAlignment(Qt.AlignCenter)
        self.layout.addWidget(self.result_label)

        self.data = None

    def load_data(self):
        file_name, _ = QFileDialog.getOpenFileName(self, "Открыть CSV", "", "CSV Files (*.csv);;All Files (*)")
        if file_name:
            try:
                self.data = pd.read_csv(file_name)
                self.display_stats()
                self.update_plot()
                self.result_label.setText("Данные успешно загружены!")
            except Exception as e:
                self.result_label.setText(f"Ошибка загрузки файла: {e}")

    def display_stats(self):
        if self.data is not None:
            numeric_data = self.data.select_dtypes(include=["number"])

            stats = f"Общая информация:\n"
            stats += f"Количество строк: {self.data.shape[0]}\n"
            stats += f"Количество столбцов: {self.data.shape[1]}\n\n"

            if not numeric_data.empty:
                stats += "Анализ числовых данных:\n"
                stats += f"Максимальные значения:\n{numeric_data.max().to_string(index=True)}\n\n"
                stats += f"Минимальные значения:\n{numeric_data.min().to_string(index=True)}\n\n"
                stats += f"Медианы:\n{numeric_data.median().to_string(index=True)}\n\n"
                stats += f"Средние значения (мат. ожидание):\n{numeric_data.mean().to_string(index=True)}\n\n"
                stats += f"Дисперсия:\n{numeric_data.var().to_string(index=True)}\n\n"
                stats += f"Стандартное отклонение:\n{numeric_data.std().to_string(index=True)}\n\n"
                stats += f"Сумма значений:\n{numeric_data.sum().to_string(index=True)}\n"
            else:
                stats += "Нет числовых данных для анализа.\n"

            self.stats_text.setText(stats)


    def update_plot(self):
        if self.data is not None:
            graph_type = self.graph_type_combo.currentText()
            self.figure.clear()
            ax = self.figure.add_subplot(111)

            try:
                if graph_type == "Линейный график":
                    ax.plot(self.data["Date"], self.data["Value1"], label="Value1", marker='o')
                    ax.set_title("Линейный график")
                    ax.set_xlabel("Дата")
                    ax.set_ylabel("Value1")
                    ax.legend()
                    plt.xticks(rotation=45)

                elif graph_type == "Гистограмма":
                    ax.bar(self.data["Date"], self.data["Value2"], label="Value2", color='skyblue')
                    ax.set_title("Гистограмма")
                    ax.set_xlabel("Дата")
                    ax.set_ylabel("Value2")
                    ax.legend()
                    plt.xticks(rotation=45)

                elif graph_type == "Круговая диаграмма":
                    category_counts = self.data["Category"].value_counts()
                    category_counts.plot.pie(
                        autopct="%1.1f%%",
                        ax=ax,
                        startangle=90,
                        counterclock=False,
                        labels=None
                    )
                    ax.set_title("Круговая диаграмма")
                    ax.set_ylabel("")
                    ax.legend(category_counts.index, bbox_to_anchor=(1, 0.5), loc="center left")

                self.canvas.draw()

            except KeyError as e:
                self.result_label.setText(f"Ошибка: отсутствует столбец {e}")


    def add_data(self):
        try:
            date = self.date_field.text()
            category = self.category_field.text()
            value1 = float(self.value1_field.text())
            value2 = float(self.value2_field.text())
            boolean_flag = self.boolean_combo.currentText() == "True"

            if all(col in self.data.columns for col in ["Date", "Category", "Value1", "Value2", "BooleanFlag"]):
                new_row = {
                    "Date": date,
                    "Category": category,
                    "Value1": value1,
                    "Value2": value2,
                    "BooleanFlag": boolean_flag
                }

                self.data = pd.concat([self.data, pd.DataFrame([new_row])], ignore_index=True)

                self.update_plot()
                self.display_stats()

                self.result_label.setText("Данные успешно добавлены!")
            else:
                self.result_label.setText("Ошибка: Необходимые столбцы отсутствуют в данных!")
        except ValueError:
            self.result_label.setText("Ошибка: Убедитесь, что все поля заполнены корректно!")


In [ ]:
if __name__ == "__main__":
    app = QApplication(sys.argv)
    main_app = MainApp()
    main_app.show()
    sys.exit(app.exec_())